In [ ]:
!pip install cufflinks plotly
!pip install chart_studio
import requests
from tqdm.auto import tqdm
import time 
import json 
import os 
import pandas as pd
import numpy as np
from datetime import datetime 
import re
from operator import itemgetter

import plotly.express as px
import plotly as py
import plotly.graph_objs as go
import chart_studio.plotly as py
from plotly.offline import iplot
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns
import cufflinks
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl', offline=True)
import altair as alt

from collections import defaultdict

Скачиваем вакансии с HH.ru, в названии которых есть слово 'Аудитор'

In [ ]:
page = 1
num_per_page = 100
test = 'NAME:Aудитор'
url = f'https://api.hh.ru/vacancies?text={test}&page={page}&per_page={num_per_page}'

In [ ]:
res = requests.get(url)
vacancies = res.json()
num_pages = vacancies.get('pages')
num_pages

In [ ]:
vacancies = res.json()
num_pages = vacancies.get('pages')
vacancy_ids = [el.get('id') for el in vacancies.get('items')]

In [ ]:
all_vacancy_ids= []
for i in tqdm(range(vacancies.get('pages'))):
    url = f'https://api.hh.ru/vacancies?text={test}&page={i}&per_page={num_per_page}'
    res = requests.get(url)
    vacancies = res.json()
    vacancy_ids = [el.get('id') for el in vacancies.get('items')]
    all_vacancy_ids.extend(vacancy_ids)

In [ ]:
len (all_vacancy_ids)

In [ ]:
vacancy_all_info = []
for j in range(10): 
    x=80*j
    j+=1
    y=80*j
    for i in range(x, y): 
        vac_id = all_vacancy_ids[i]
        url = f'https://api.hh.ru/vacancies/{vac_id}'
        res = requests.get(url)
        vacancy = res.json()
        vacancy_all_info.append(vacancy)
    print(j)
    time.sleep(180) 

In [ ]:
vacancy = pd.DataFrame(vacancy_all_info, columns = ['id', 'name','area','salary','description','experience', 'schedule', 'employment', 'employer', 'key_skills', 
                                                      'published_at' ])

Очищаем датафрейм для анализа данных

In [ ]:
#выведем значение словаря 'name' через функцию python
def name (dictionary):
    name_dictionary = dictionary ['name']
    return name_dictionary


In [ ]:
vacancy['area'] = vacancy['area'].apply(name)
vacancy['experience'] = vacancy['experience'].apply(name)
vacancy['schedule'] = vacancy['schedule'].apply(name)
vacancy['employment'] = vacancy['employment'].apply(name)
vacancy['employer'] = vacancy['employer'].apply(name)

In [ ]:
#выведем значение словаря по полю salary from через функцию python
def salary_from (dictionary):
    if dictionary is None:
        return None 
    else:
         return dictionary ['from']

In [ ]:
#выведем значение словаря по полю salary to через функцию python
def salary_to (dictionary):
    if dictionary is None:
        return None
    else:
        return dictionary ['to']

In [ ]:
#выведем значение словаря по полю salary to через функцию python
def salary_to (dictionary):
    if dictionary is None:
        return None
    else:
        return dictionary ['to']

In [ ]:
#выведем значение словаря по полю salary to через функцию python
def currency (dictionary):
    if dictionary is None:
        return None
    else:
        return dictionary ['currency']

In [ ]:
vacancy['salary_from'] = vacancy['salary'].apply(salary_from)
vacancy['salary_to'] = vacancy['salary'].apply(salary_to)
vacancy['currency'] = vacancy['salary'].apply(currency)

In [ ]:
#удаляем колонку salary, т.к. она больше не нужна
vacancy = vacancy.drop(columns=['salary'])

In [ ]:
#переводим поле published_at в формат datetime
vacancy['published_at'] = pd.to_datetime(vacancy['published_at'])

In [ ]:
#выведем ключевые навыки для аудиторо в отдельный датафрейм
skills = []
for i in range(len(df_vacancy)): 
    if len(vacancy['key_skills'][i]) != 0: 
        x=vacancy['key_skills'][i]
        for j in range(len(x)): 
            skills.append(x[j]['name'])

In [ ]:
auditor_skills = pd.DataFrame(skills, columns = ['name_skill'])
auditor_skills

In [ ]:
#запишем вакансии в файл csv
vacancy.to_csv("вакансии_аудитор.csv")

Исследование вакансий аудиторов

In [ ]:
#график публикации вакансий
publ_at = vacancy.groupby(['published_at']).count()
publ_at_count = pd.DataFrame(publ_at, columns = ['id'])
publ_at_count.plot()

In [ ]:
#выведем топ-15 скиллов, необходимых аудиторам
df = auditor_skills.value_counts().head(15).rename_axis('unique_values').to_frame('counts')
print (df)

In [ ]:
df.groupby(['unique_values']).sum().plot(kind='pie', y='counts', autopct='%1.0f%%', legend=False)

Из диаграммы видно, что ключевыми навыками для аудиторов является знание основ аудита, опыт в проведении аудиторских проверок. Из личностных качеств - ответственность, умение работать в команде, грамотная речь и мобильность. 

In [ ]:
#соберем в отдельный датафрейм топ-15 работодателей, разместивших вакансии аудиторов
top_15 = pd.DataFrame((vacancy.groupby('employer')[['id']].agg({'id': 'count'})
          .sort_values(by='id', ascending=False))[:15].reset_index())

In [ ]:
top_15

In [ ]:
fig, ax = plt.subplots()
top_15.plot(kind='barh', y="id", x="employer", ax=ax)

Наибольший процент вакансии принадлежит компании GLOBALTEAM RUSSIA, которая предоставляет услуги:все виды мерчандайзинга;
полевой аудит торговых точек на предмет соблюдения контрактных обязательств по представленности товаров и пр.;
исследования, направленные на оценку потребительского опыта, полученного клиентом в процессе приобретения товара или услуги (тайный покупатель, тайный гость и пр.);мониторинг и анализ цен конкурентов;маркетинговые активности.
СБЕР в поиске 12 сотрудников на должность аудитора. 

In [ ]:
vacancy['currency'].value_counts()

In [ ]:
#выберем вакансии только в рублях
rur_vac = vacancy.dropna()[vacancy.dropna()['currency'].str.contains("RUR")]
rur_vac.head()

In [ ]:
#определим уровень заработной платы аудиторов в зависимости от опыта работы
rur_vac.pivot(columns='experience', values='salary_to').iplot(
        kind='box',
        yTitle='salary',
        title='Уровень зарплаты в зависимости от опыта работы')

Можно сделать вывод, что самый низкий уровень заработной платы у аудиторов без опыта работы. Максимальный доход получают специалисты с опытом от 3х лет и выше.

In [ ]:
rur_vac.iplot(
    x='area',
    y='salary_to',
    # Specify the category
    categories='employment',
    xTitle='Город',
    yTitle='Уровень зарплаты',
    title='Уровень заработной платы аудиторов в зависимости от типа занятости в городах России')

Наиболее высокооплачиваемыми являются вакансии с полной занятостью. 

In [ ]:
top_city_vac = rur_vac.loc[rur_vac['area'].isin(['Самара','Москва','Санкт-Петербург','Екатеринбург','Новосибирск','Казань',
                                                'Нижний Новгород', 'Краснодар', 'Уфа'])]
                         

In [ ]:
#Средние зарплаты аудиторов в рублях в крупнейших городах России
alt.Chart(top_city_vac).mark_bar().encode(

    y='mean(salary_to):Q',
    x='area:N'
)

In [ ]:
#размер заработной платы в зависимости от города и компании-работодателя

pivot = top_city_vac.pivot_table(
    index=["area"],
    columns=["employer"],
    values="salary_to",
    aggfunc=np.average)
sns.heatmap(pivot) # 

In [ ]:
#выберем вакансии СБЕРа
sber_vac = vacancy[vacancy['employer'].str.contains('СБЕР')]
sber_vac

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import f1_score
import nltk
import ssl
from nltk.corpus import stopwords
import re
import pymorphy2

In [ ]:
sber_vac_2 = pd.DataFrame(sber_vac, columns = ['description'])
sber_vac_2 

In [ ]:
RANDOM_STATE = 222

In [ ]:
text_transformer = TfidfVectorizer()

In [ ]:
text = text_transformer.fit_transform(sber_vac_2['description'])

In [ ]:
text

In [ ]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

In [ ]:
nltk.download('stopwords')

In [ ]:
stop_russian = stopwords.words('russian')

In [ ]:
text_transformer2 = TfidfVectorizer(stop_words=stop_russian, 
                                   ngram_range=(1,1), 
                                   lowercase=True)

In [ ]:
def remove_trash(list): 
    pattern = r'[^А-Яа-я]+'
    try:
      list = [re.sub(pattern, ' ', i) for i in list] 
#       print (list)
    except Exception as e:
      print(e)
    return list

In [ ]:
sber_vac_2['text_clean'] = remove_trash(sber_vac_2['description'])

In [ ]:
sber_vac_2

In [ ]:
morph = pymorphy2.MorphAnalyzer()

In [ ]:
def lemmatize(row):
    t = []
    text = row['text_clean']
    for word in text.split():
        if len(word)<=1:
            continue
        p = morph.parse(word)[0]
        t.append(p.normal_form)
    return " ".join(t)

In [ ]:
sber_vac_2['text_clean_norm'] = sber_vac_2.apply(lemmatize,axis=1)

In [ ]:
sber_vac_2

In [ ]:
from collections import Counter

vacancy_descr = sber_vac_2.text_clean_norm 
cloud = Counter(vacancy_descr)
from wordcloud import WordCloud, STOPWORDS
stopwords = set(STOPWORDS)
cloud = ''
for x in list(vacancy_descr):
    cloud+=x+' '
wordcloud = WordCloud(width = 800, height = 800, 

                stopwords = stopwords, 
                min_font_size = 8,background_color='white'
                     ).generate(cloud)

import matplotlib.pylab as plt
plt.figure(figsize = (16, 16)) 
plt.imshow(wordcloud)
plt.savefig('vacancy_cloud.png')
